In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import pandas as pd
import numpy as np


In [3]:
train_df = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/train.csv')
train_df.head(10)

,bant_submit,customer_country,business_unit,com_reg_ver_win_rate,customer_idx,customer_type,enterprise,historical_existing_cnt,id_strategic_ver,it_strategic_ver,...,response_corporate,expected_timeline,ver_cus,ver_pro,ver_win_rate_x,ver_win_ratio_per_bu,business_area,business_subarea,lead_owner,is_converted
0,1.00,/Quezon City/Philippines,AS,0.066667,32160,End-Customer,Enterprise,NaN,NaN,NaN,...,LGEPH,less than 3 months,1,0,0.003079,0.026846,corporate / office,Engineering,0,True
1,1.00,/PH-00/Philippines,AS,0.066667,23122,End-Customer,Enterprise,12.0,NaN,NaN,...,LGEPH,less than 3 months,1,0,0.003079,0.026846,corporate / office,Advertising,1,True
2,1.00,/Kolkata /India,AS,0.088889,1755,End-Customer,Enterprise,144.0,NaN,NaN,...,LGEIL,less than 3 months,1,0,0.003079,0.026846,corporate / office,Construction,2,True
3,1.00,/Bhubaneswar/India,AS,0.088889,4919,End-Customer,Enterprise,NaN,NaN,NaN,...,LGEIL,less than 3 months,1,0,0.003079,0.026846,corporate / office,IT/Software,3,True
4,1.00,/Hyderabad/India,AS,0.088889,17126,Specifier/ Influencer,Enterprise,NaN,NaN,NaN,...,LGEIL,less than 3 months,0,0,0.003079,0.026846,corporate / office,NaN,4,True
5,1.00,/Abuja/Nigeria,AS,0.040816,16328,End-Customer,SMB,NaN,NaN,NaN,...,LGEAF,less than 3 months,1,0,0.003079,0.026846,corporate / office,Engineering,5,True
6,0.75,"/Jeddah, KSA/Saudi Arabia",AS,0.040816,20664,End-Customer,SMB,NaN,NaN,NaN,...,LGESJ,NaN,1,0,0.003079,0.026846,corporate / office,Engineering,6,True
7,1.00,/Guwahati/India,AS,0.088889,17983,End-Customer,SMB,NaN,NaN,NaN,...,LGEIL,less than 3 months,1,0,0.003079,0.026846,corporate / office,Manufacturing,7,True
8,0.75,/Cebu city/Philippines,AS,0.066667,30867,Specifier/ Influencer,Enterprise,3.0,NaN,NaN,...,LGEPH,less than 3 months,0,0,0.003079,0.026846,corporate / office,Construction,8,True
9,0.75,"/hauz khas,delhi/India",AS,0.088889,6084,End-Customer,SMB,NaN,NaN,NaN,...,LGEIL,3 months ~ 6 months,1,0,0.003079,0.026846,corporate / office,NaN,9,True


In [7]:
train_df.info()    # null 값이 존재하는 열 많음! (59,299 아니면 모두 null 값 존재하는것)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 59299 entries, 0 to 59298
Data columns (total 29 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   bant_submit              59299 non-null  float64
 1   customer_country         58317 non-null  object 
 2   business_unit            59299 non-null  object 
 3   com_reg_ver_win_rate     14568 non-null  float64
 4   customer_idx             59299 non-null  int64  
 5   customer_type            15338 non-null  object 
 6   enterprise               59299 non-null  object 
 7   historical_existing_cnt  13756 non-null  float64
 8   id_strategic_ver         3444 non-null   float64
 9   it_strategic_ver         1121 non-null   float64
 10  idit_strategic_ver       4565 non-null   float64
 11  customer_job             40566 non-null  object 
 12  lead_desc_length         59299 non-null  int64  
 13  inquiry_type             58358 non-null  object 
 14  product_category      

In [8]:
# train 데이터에 있는 column별 category 값 확인
print(train_df.columns.values)

['bant_submit' 'customer_country' 'business_unit' 'com_reg_ver_win_rate'
 'customer_idx' 'customer_type' 'enterprise' 'historical_existing_cnt'
 'id_strategic_ver' 'it_strategic_ver' 'idit_strategic_ver' 'customer_job'
 'lead_desc_length' 'inquiry_type' 'product_category'
 'product_subcategory' 'product_modelname' 'customer_country.1'
 'customer_position' 'response_corporate' 'expected_timeline' 'ver_cus'
 'ver_pro' 'ver_win_rate_x' 'ver_win_ratio_per_bu' 'business_area'
 'business_subarea' 'lead_owner' 'is_converted']


## **bant_submit**

MQL 구성 요소들 중 아래 4가지 항목에 대해 작성된 값의 비율
*   Budget (예산)
*   Title (고객의 직책/직급)
*   Needs (요구사항)
*   Timeline (희망 납기일)

In [9]:
# bant_submit (합산: 59,299 확인!)
# 1로 가까울수록 좋은건지? - 확인필요
train_df['bant_submit'].value_counts()

0.50    17192
1.00    17061
0.25    12852
0.75    11681
0.00      513
Name: bant_submit, dtype: int64

## **customer_country**
고객의 국적을 나타낸다.
*  /... /도시2/도시1/나라명
*  default 값: //나라명
*  도시가 여러개 들어갈 수도 있음 → 쉼표로 구분  e.g. /Tirurangadi, Malappuram, Kerala/India)

In [11]:
# 나라별, 지역별로 나눴을 때 구매까지 이어지는지를 볼 수 있겠음
## 나라만 볼지, 지역까지 세부적으로 들어가서 볼지, 비교하면서 결정하는 방법!
train_df['customer_country'].value_counts()

//India                                   3037
/São Paulo/Brazil                         1344
//United States                            962
//United Kingdom                           805
//Saudi Arabia                             718
                                          ... 
/Hydrated/India                              1
/Tirurangadi, Malappuram, Kerala/India       1
/Canoinhas / SC/Brazil                       1
/Tupã/SP/Brazil                              1
/paris/France                                1
Name: customer_country, Length: 15399, dtype: int64

In [25]:
# 해당 열에 null 값이 하나라도 존재하면 True를 반환 → null 처리필요!!
has_null_values = train_df['customer_country'].isnull().any()
print(has_null_values)

null_count = train_df['customer_country'].isnull().sum()
print(f"Null 값의 개수: {null_count}")

True
Null 값의 개수: 982


In [20]:
# 맨 뒤 부분이 나라명인줄 알았으나, 약 200위권 부터는 이상한 값이 들어가 있음
## 전체 category 종류는 582개!

country = train_df['customer_country'].str.split('/').str[-1].value_counts().head(200)
print(country)

India                               16578
Brazil                               8676
United States                        3767
Mexico                               2749
Philippines                          2538
                                    ...  
 South Africa                           2
 Türkiye                                2
MI 48827                                2
300 East Park Drive                     2
6564 Headquarters DrPlanoTX75051        2
Name: customer_country, Length: 200, dtype: int64


## **business_unit**
MQL 요청 상품에 대응되는 사업부 (각각 무슨 부서를 뜻하는지 확인필요)
*  ID
*  AS
*  IT
*  Solution
*  CM


In [12]:
# 합산값 59,299 확인!

train_df['business_unit'].value_counts()

ID          25563
AS          24774
IT           8664
Solution      296
CM              2
Name: business_unit, dtype: int64

## **com_reg_ver_win_rate**
Vertical Level 1, business unit, region을 기준으로 oppty 비율을 계산
(*oppty: 얼마나 많은 유효 딜을 관리하고 있는가)

In [29]:
train_df['com_reg_ver_win_rate'].value_counts()

0.074949    1130
0.003937     803
0.073248     791
0.642857     782
0.003788     713
            ... 
0.363636      13
0.444444      12
0.400000      12
1.000000      10
0.200000       5
Name: com_reg_ver_win_rate, Length: 80, dtype: int64

In [31]:
# nulll 값 매우많음 ⇒ 단순히 0이 아니라, null값을 의미있게 채워야 하지 않을까?
has_null_values = train_df['com_reg_ver_win_rate'].isnull().any()
print(has_null_values)

null_count = train_df['com_reg_ver_win_rate'].isnull().sum()
print(f"Null 값의 개수: {null_count}")

True
Null 값의 개수: 44731


## **customer_type**

In [32]:
# non-null : 15,338개 밖에 안됨 ⇒ null값 매우 많음
train_df['customer_type'].value_counts()

End-Customer                    6647
End Customer                    3996
Specifier/ Influencer           2525
Channel Partner                 1368
Service Partner                  349
Solution Eco-Partner             146
Installer/Contractor              52
Specifier / Influencer            43
Corporate                         31
HVAC Engineer                     23
Engineer                          20
Developer                         17
Technician                        16
Consultant                        15
Other                             10
Home Owner                        10
End-user                           8
Manager / Director                 8
Software/Solution Provider         7
Etc.                               6
Architect/Consultant               5
Homeowner                          5
Reseller                           5
Installer                          5
Interior Designer                  5
Distributor                        4
Others                             4
D

## **enterprise**
* Global 기업: Enterprise
* Small/Medium 기업: SMB

In [33]:
# null값 없음
train_df['enterprise'].value_counts()

Enterprise    37783
SMB           21516
Name: enterprise, dtype: int64

## **historical_existing_cnt**
이전에 Converted(영업 전환) 되었던 횟수 (max_vlaue: 1394.0, min_value: 0.0)

In [34]:
train_df['historical_existing_cnt'].value_counts()

4.0       2957
0.0       1973
1.0       1967
19.0      1441
16.0      1132
          ... 
404.0        1
145.0        1
1394.0       1
199.0        1
108.0        1
Name: historical_existing_cnt, Length: 136, dtype: int64

In [36]:
max_value = train_df['historical_existing_cnt'].max()
min_value = train_df['historical_existing_cnt'].min()

print(f'max_vlaue: {max_value}, min_value: {min_value}')

max_vlaue: 1394.0, min_value: 0.0


## **id_strategic_ver / it_strategic_ver / idit_strategic_ver**
* id_strategic_ver : 특정 사업부(Business Unit), 특정 사업 영역(Vertical Level1)에 대해 가중치를 부여  [3,444개]
* it_strategic_ver : 특정 사업부(Business Unit), 특정 사업 영역(Vertical Level1)에 대해 가중치를 부여  [1,121개]
* idit_strategic_ver : id_strategic_ver or it_strategic_ver 값 중
하나라도 1의 값을 가지면 1 값으로 표현  [4,565개]

In [37]:
# 3가지 변수모두 값은 0 아니면 null값 이다.
## id & it = idit

id_strategic_ver = train_df['id_strategic_ver'].value_counts()
it_strategic_ver = train_df['it_strategic_ver'].value_counts()
idit_strategic_ver = train_df['idit_strategic_ver'].value_counts()

In [40]:
idit_strategic_ver

1.0    4565
Name: idit_strategic_ver, dtype: int64

## **customer_job**
고객의 직업군

In [42]:
# 560가지 종류가 존재 (Other...?)
## non-null : 40,566개
train_df['customer_job'].value_counts()

engineering                       6342
other                             4852
administrative                    3359
education                         2287
sales                             2218
                                  ... 
appliance specialist                 1
av estimator                         1
purchasers                           1
pricing                              1
exhibition / convention center       1
Name: customer_job, Length: 560, dtype: int64

## **lead_desc_length**
고객이 작성한 Lead Descriptoin 텍스트 총 길이

In [46]:
# null값 없음!!
train_df['lead_desc_length'].value_counts().sort_index(ascending=False)

1264        1
1198        1
1181        1
1174        2
1168        1
        ...  
5         818
4         884
3       10378
2         934
1         771
Name: lead_desc_length, Length: 909, dtype: int64

In [50]:
# 빈도수가 큰 순서대로 봐보자
train_df['lead_desc_length'].value_counts(ascending=False).head(500)

3      10378
14      3180
2        934
10       891
4        884
       ...  
555        6
514        6
513        6
453        6
439        6
Name: lead_desc_length, Length: 500, dtype: int64

## **inquiry_type**
고객의 문의유형

* Quotation or purchase consultation / Quotation or Purchase Consultation 두개 통합 필요함! (C 소문자, 대문자 차이임)

In [51]:
# non-null : 58358
train_df['inquiry_type'].value_counts()

Quotation or purchase consultation                                                                                     23274
Quotation or Purchase Consultation                                                                                     18766
Sales Inquiry                                                                                                           9977
Product Information                                                                                                     1237
Other                                                                                                                    929
                                                                                                                       ...  
quotation_                                                                                                                 1
display product                                                                                                            1


## **product_category / product_subcategory / product_modelname**
* 'product_category': 요청 제품 카테고리 [39,925개]
* 'product_subcategory': 요청 제품 하위 카테고리 [9,235개]
* 'product_modelname': 요청 제품 모델명 [9,229개]

> 3개를 합치는 방법이 없을까?



In [52]:
product_category = train_df['product_category'].value_counts()
product_subcategory = train_df['product_subcategory'].value_counts()
product_modelname = train_df['product_modelname'].value_counts()

In [56]:
# product_category ⇒ 빈도수가 큰 순서
train_df['product_category'].value_counts(ascending=False).head(50)

interactive digital board            6154
vrf                                  5640
multi-split                          3674
video wall signage                   2029
etc.                                 2018
led signage                          1895
interactive signage                  1862
single-split                         1569
rac                                  1447
oled signage                         1350
hotel tv                             1112
chiller                              1051
standard signage                      996
medical display                       962
lg one:quick series                   768
monitor                               762
one:quick series                      668
heating                               655
high brightness signage               477
ventilation                           420
teto ou cassete inverter              308
control                               263
multi inverter                        257
ar condicionado residencial       

In [61]:
# product_subcategory ⇒ 빈도수가 큰 순서
train_df['product_subcategory'].value_counts(ascending=False).head(50)

All Medical Displays                             446
Interactive Digital Board                        417
TR3BF Series                                     374
Transparent OLED Signage                         288
Diagnostic Monitors                              284
Transparent LED Film                             280
49" 500 nits FHD Slim Bezel Video Wall           268
TR3DJ Series                                     246
One:Quick Flex                                   243
One:Quick Works                                  228
55" 500 nits FHD 0.44mm Even Bezel Video Wall    209
55" 700 nits FHD 0.44mm Even Bezel Video Wall    180
Digital X-ray Detectors                          167
UH5F-H Series                                    156
US660H Series                                    151
4K UHD Hospitality TV with Pro:Centric Direct    127
VL5F Series                                      124
Outdoor Display                                  120
LG MAGNIT                                     

In [62]:
# product_modelname ⇒ 빈도수가 큰 순서
train_df['product_modelname'].value_counts(ascending=False).head(50)

UltraFine Ergo(32UN880)     523
28MQ780                     479
All Medical Displays        350
43HT3WJ                     290
55CT5WJ                     261
55VSM5J                     199
Ergo Dual(27QP88D)          167
Diagnostic Monitors         167
75TC3D                      159
55VSH7J                     152
LAT140                      152
86TR3E                      143
DualUp(28MQ780)             139
21HQ513D                    132
UltraWide Ergo(34WN780)     117
49VL5G-M                    115
Digital X-ray Detectors     104
49VL5G                       94
86TR3DJ                      94
LAT240                       93
75TR3DJ                      88
SC-00DA                      88
65EP5G                       85
55TC3D                       83
55EW5G-V                     77
55EW5TK-A                    77
55VL5F-A                     76
86TR3BF-B                    73
65TR3DJ                      72
LSAB009                      72
75TR3BF-B                    70
65TR3BF-

## **customer_country.1**
담당 자사 법인명 기반의 지역 정보(대륙)   [58,317개]


> 'customer_country' 컬럼과 유사도가 얼마나 있는지 확인 필요 (*텍스트 쓰는 방식도 같은 느낌임)



In [63]:
train_df['customer_country.1'].value_counts()

//India                                   3037
/São Paulo/Brazil                         1344
//United States                            962
//United Kingdom                           805
//Saudi Arabia                             718
                                          ... 
/Hydrated/India                              1
/Tirurangadi, Malappuram, Kerala/India       1
/Canoinhas / SC/Brazil                       1
/Tupã/SP/Brazil                              1
/paris/France                                1
Name: customer_country.1, Length: 15399, dtype: int64

## **customer_position**
고객의 회사 직책 (null값 없음!)

In [64]:
# none ⇒ 19,680 !!
## none값은 제외가 필요할 듯
train_df['customer_position'].value_counts()

none                      19680
manager                    8217
ceo/founder                7984
other                      6499
director                   4843
                          ...  
chairman                      1
pgt physics                   1
education professional        1
chemistry teacher             1
radiology professional        1
Name: customer_position, Length: 117, dtype: int64

## **response_corporate**
담당 자사 법인명 (null값 없음!!!)
> 각 알파벳이 의미하는 바가 있을 듯 ⇒ 알파벳을 나누어서 기준을 잡아야하지 않을까?

In [65]:
train_df['response_corporate'].value_counts()

LGEIL    16908
LGESP     9311
LGEUS     5955
LGEMS     2768
LGEPH     2651
LGEGF     2149
LGECB     2079
LGEUK     1651
LGESJ     1469
LGECL     1339
LGEPS     1252
LGEIS     1146
LGEPR     1131
LGEDG     1051
LGEPL      850
LGEEG      704
LGEVH      614
LGEES      592
LGETK      541
LGEAR      491
LGEKR      433
LGEHK      383
LGEAP      366
LGESL      348
LGEMK      331
LGEFS      302
LGEAF      288
LGEIN      281
LGELF      274
LGESA      268
LGECI      263
LGETH      185
LGEEF      159
LGEPT      111
LGEML      110
LGEBN       99
LGEYK       78
LGECH       65
LGEHS       51
LGETT       47
LGEJP       41
LGEAS       33
LGESW       31
LGEMC       30
LGERO       29
LGEEB       13
LGEAG        8
LGERA        8
LGECZ        7
LGELA        2
LGEUR        1
LGEIR        1
LGEBT        1
Name: response_corporate, dtype: int64

## **expected_timeline**
고객의 요청한 처리 일정

In [67]:
train_df['expected_timeline'].value_counts(ascending=False).head(30)

less than 3 months                                                                                                                                             17250
3 months ~ 6 months                                                                                                                                             5026
more than a year                                                                                                                                                3023
9 months ~ 1 year                                                                                                                                               1101
6 months ~ 9 months                                                                                                                                             1098
less than 6 months                                                                                                                                               108
etc.      

## ver_cus / ver_pro
* 'ver_cus' : 특정 Vertical Level 1(사업영역) and Customer_type(고객 유형)이 소비자(End-user)인 경우에 대한 가중치
* 'ver_pro' : 특정 Vertical Level 1(사업영역) and 특정 Product Category(제품 유형)인 경우에 대한 가중치

In [4]:
# null값 없음
# train_df['customer_type'] => End-Customer (6647) / End Customer (3996)
train_df['ver_cus'].value_counts()

0    56832
1     2467
Name: ver_cus, dtype: int64

In [9]:
# 'End-Customer', 'End Customer' 로 했을 때, 결과가 이상하게 나옴
selected_rows = train_df[train_df['customer_type'].isin(['End-Customer', 'End Customer'])]
result_df = selected_rows[['customer_type', 'ver_cus']]

result_df

,customer_type,ver_cus
0,End-Customer,1
1,End-Customer,1
2,End-Customer,1
3,End-Customer,1
5,End-Customer,1
...,...,...
59284,End-Customer,0
59285,End-Customer,0
59286,End Customer,0
59293,End Customer,0


In [10]:
train_df['ver_pro'].value_counts()

0    56286
1     3013
Name: ver_pro, dtype: int64

In [13]:
# 가중치를 준 값들의 'product_category'가 무엇인지 확인!
selected_rows = train_df[train_df['ver_pro'] == 1]
result_df = selected_rows[['ver_pro', 'product_category']]

result_df['product_category'].value_counts()

video wall signage                                                                                      667
led signage                                                                                             641
interactive signage                                                                                     406
oled signage                                                                                            390
standard signage                                                                                        325
hotel tv                                                                                                259
high brightness signage                                                                                 205
special signage                                                                                          47
signage care solution                                                                                    17
monitor signage,monior/monit

## ver_win_ratio_per_bu
* 특정 Vertical Level1의 Business Unit 별 샘플 수 대비
영업 전환된 샘플 수의 비율을 계산

In [14]:
# 'historical_existing_cnt' = 이전에 Converted(영업 전환) 되었던 횟수
## vertical level1 이 뭘까..?
train_df['ver_win_ratio_per_bu'].value_counts()

0.064566    2460
0.049840    2028
0.020121    1365
0.048630    1356
0.026846    1278
0.064070    1075
0.071345     984
0.011583     788
0.022634     713
0.060924     540
0.079412     421
0.036913     403
0.035484     385
0.031579     271
0.053571     216
0.051471     210
0.028777     200
0.022727     173
0.128571     156
0.131148     148
0.227273      62
0.285714      43
0.034483      29
Name: ver_win_ratio_per_bu, dtype: int64

## business_area / business_subarea
* 'business_area' : 고객의 사업 영역
* 'business_subarea' : 고객의 세부 사업 영역

In [15]:
train_df['business_area'].value_counts()

corporate / office                4097
retail                            3011
hotel & accommodation             2013
special purpose                   1929
residential (home)                1818
education                         1727
hospital & health care            1199
factory                           1035
government department              650
public facility                    519
transportation                     306
power plant / renewable energy     113
Name: business_area, dtype: int64

In [16]:
# others가 너무 많은데..?
train_df['business_subarea'].value_counts()

Others                           1740
Other Stores                      280
Hospital                          267
Manufacturing Factory / Plant     196
Construction                      191
                                 ... 
Duty Free Shop                      2
Dormitory                           1
Casino Resort                       1
Holdings                            1
Travel Agency                       1
Name: business_subarea, Length: 86, dtype: int64

## lead_owner
영업 담당자 이름

In [17]:
# 각 숫자가 무엇과 관련있는지 확인필요
train_df['lead_owner'].value_counts()

174    4627
175    4120
97     2499
23     2453
194    2208
       ... 
481       1
909       1
979       1
111       1
756       1
Name: lead_owner, Length: 984, dtype: int64